In [1]:
# !git clone https://github.com/facebookresearch/detr.git

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
# from tqdm.autonotebook import tqdm
from tqdm import tqdm
import json
import collections

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

#sklearn
from sklearn.model_selection import StratifiedKFold

#CV
import cv2

#Glob
from glob import glob

In [4]:
import sys
if '../../albumentations' not in sys.path:
    sys.path.append('../../albumentations')

In [5]:
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2 

In [6]:
#Albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

In [7]:
# ################# DETR FUCNTIONS FOR LOSS######################## 
# import sys
# sys.path.append('./detr/')

# from detr.models.matcher import HungarianMatcher
# from detr.models.detr import SetCriterion
# #################################################################

In [8]:
# Reading annotations.json
TRAIN_ANNOTATIONS_PATH = "../resource/train/ann_train_without_error.json"
TRAIN_IMAGE_DIRECTIORY = "D:\\data\\food_detection\\public_train\\images"

VAL_ANNOTATIONS_PATH = "../resource/val/ann_valid.json"
VAL_IMAGE_DIRECTIORY = "D:\\data\\food_detection\\public_validation\\images"

In [9]:
CLASSES = ('bread-wholemeal', 'jam', 'water', 'bread-sourdough', 'banana', 'soft-cheese', 'ham-raw', 'hard-cheese', 'cottage-cheese', 'bread-half-white', 'coffee-with-caffeine', 'fruit-salad', 'pancakes', 'tea', 'salmon-smoked', 'avocado', 'spring-onion-scallion', 'ristretto-with-caffeine', 'ham', 'egg', 'bacon-frying', 'chips-french-fries', 'juice-apple', 'chicken', 'tomato-raw', 'broccoli', 'shrimp-boiled', 'beetroot-steamed-without-addition-of-salt', 'carrot-raw', 'chickpeas', 'french-salad-dressing', 'pasta-hornli', 'sauce-cream', 'meat-balls', 'pasta', 'tomato-sauce', 'cheese', 'pear', 'cashew-nut', 'almonds', 'lentils', 'mixed-vegetables', 'peanut-butter', 'apple', 'blueberries', 'cucumber', 'cocoa-powder', 'greek-yaourt-yahourt-yogourt-ou-yoghourt', 'maple-syrup-concentrate', 'buckwheat-grain-peeled', 'butter', 'herbal-tea', 'mayonnaise', 'soup-vegetable', 'wine-red', 'wine-white', 'green-bean-steamed-without-addition-of-salt', 'sausage', 'pizza-margherita-baked', 'salami', 'mushroom', 'bread-meat-substitute-lettuce-sauce', 'tart', 'tea-verveine', 'rice', 'white-coffee-with-caffeine', 'linseeds', 'sunflower-seeds', 'ham-cooked', 'bell-pepper-red-raw', 'zucchini', 'green-asparagus', 'tartar-sauce', 'lye-pretzel-soft', 'cucumber-pickled', 'curry-vegetarian', 'yaourt-yahourt-yogourt-ou-yoghourt-natural', 'soup-of-lentils-dahl-dhal', 'soup-cream-of-vegetables', 'balsamic-vinegar', 'salmon', 'salt-cake-vegetables-filled', 'bacon', 'orange', 'pasta-noodles', 'cream', 'cake-chocolate', 'pasta-spaghetti', 'black-olives', 'parmesan', 'spaetzle', 'salad-lambs-ear', 'salad-leaf-salad-green', 'potatoes-steamed', 'white-cabbage', 'halloumi', 'beetroot-raw', 'bread-grain', 'applesauce-unsweetened-canned', 'cheese-for-raclette', 'mushrooms', 'bread-white', 'curds-natural-with-at-most-10-fidm', 'bagel-without-filling', 'quiche-with-cheese-baked-with-puff-pastry', 'soup-potato', 'bouillon-vegetable', 'beef-sirloin-steak', 'taboule-prepared-with-couscous', 'eggplant', 'bread', 'turnover-with-meat-small-meat-pie-empanadas', 'mungbean-sprouts', 'mozzarella', 'pasta-penne', 'lasagne-vegetable-prepared', 'mandarine', 'kiwi', 'french-beans', 'tartar-meat', 'spring-roll-fried', 'pork-chop', 'caprese-salad-tomato-mozzarella', 'leaf-spinach', 'roll-of-half-white-or-white-flour-with-large-void', 'pasta-ravioli-stuffing', 'omelette-plain', 'tuna', 'dark-chocolate', 'sauce-savoury', 'dried-raisins', 'ice-tea', 'kaki', 'macaroon', 'smoothie', 'crepe-plain', 'chicken-nuggets', 'chili-con-carne-prepared', 'veggie-burger', 'cream-spinach', 'cod', 'chinese-cabbage', 'hamburger-bread-meat-ketchup', 'soup-pumpkin', 'sushi', 'chestnuts', 'coffee-decaffeinated', 'sauce-soya', 'balsamic-salad-dressing', 'pasta-twist', 'bolognaise-sauce', 'leek', 'fajita-bread-only', 'potato-gnocchi', 'beef-cut-into-stripes-only-meat', 'rice-noodles-vermicelli', 'tea-ginger', 'tea-green', 'bread-whole-wheat', 'onion', 'garlic', 'hummus', 'pizza-with-vegetables-baked', 'beer', 'glucose-drink-50g', 'chicken-wing', 'ratatouille', 'peanut', 'high-protein-pasta-made-of-lentils-peas', 'cauliflower', 'quiche-with-spinach-baked-with-cake-dough', 'green-olives', 'brazil-nut', 'eggplant-caviar', 'bread-pita', 'pasta-wholemeal', 'sauce-pesto', 'oil', 'couscous', 'sauce-roast', 'prosecco', 'crackers', 'bread-toast', 'shrimp-prawn-small', 'panna-cotta', 'romanesco', 'water-with-lemon-juice', 'espresso-with-caffeine', 'egg-scrambled-prepared', 'juice-orange', 'ice-cubes', 'braided-white-loaf', 'emmental-cheese', 'croissant-wholegrain', 'hazelnut-chocolate-spread-nutella-ovomaltine-caotina', 'tomme', 'water-mineral', 'hazelnut', 'bacon-raw', 'bread-nut', 'black-forest-tart', 'soup-miso', 'peach', 'figs', 'beef-filet', 'mustard-dijon', 'rice-basmati', 'mashed-potatoes-prepared-with-full-fat-milk-with-butter', 'dumplings', 'pumpkin', 'swiss-chard', 'red-cabbage', 'spinach-raw', 'naan-indien-bread', 'chicken-curry-cream-coconut-milk-curry-spices-paste', 'crunch-muesli', 'biscuits', 'bread-french-white-flour', 'meatloaf', 'fresh-cheese', 'honey', 'vegetable-mix-peas-and-carrots', 'parsley', 'brownie', 'dairy-ice-cream', 'tea-black', 'carrot-cake', 'fish-fingers-breaded', 'salad-dressing', 'dried-meat', 'chicken-breast', 'mixed-salad-chopped-without-sauce', 'feta', 'praline', 'tea-peppermint', 'walnut', 'potato-salad-with-mayonnaise-yogurt-dressing', 'kebab-in-pita-bread', 'kolhrabi', 'alfa-sprouts', 'brussel-sprouts', 'bacon-cooking', 'gruyere', 'bulgur', 'grapes', 'pork-escalope', 'chocolate-egg-small', 'cappuccino', 'zucchini-stewed-without-addition-of-fat-without-addition-of-salt', 'crisp-bread-wasa', 'bread-black', 'perch-fillets-lake', 'rosti', 'mango', 'sandwich-ham-cheese-and-butter', 'muesli', 'spinach-steamed-without-addition-of-salt', 'fish', 'risotto-without-cheese-cooked', 'milk-chocolate-with-hazelnuts', 'cake-oblong', 'crisps', 'pork', 'pomegranate', 'sweet-corn-canned', 'flakes-oat', 'greek-salad', 'cantonese-fried-rice', 'sesame-seeds', 'bouillon', 'baked-potato', 'fennel', 'meat', 'bread-olive', 'croutons', 'philadelphia', 'mushroom-average-stewed-without-addition-of-fat-without-addition-of-salt', 'bell-pepper-red-stewed-without-addition-of-fat-without-addition-of-salt', 'white-chocolate', 'mixed-nuts', 'breadcrumbs-unspiced', 'fondue', 'sauce-mushroom', 'tea-spice', 'strawberries', 'tea-rooibos', 'pie-plum-baked-with-cake-dough', 'potatoes-au-gratin-dauphinois-prepared', 'capers', 'vegetables', 'bread-wholemeal-toast', 'red-radish', 'fruit-tart', 'beans-kidney', 'sauerkraut', 'mustard', 'country-fries', 'ketchup', 'pasta-linguini-parpadelle-tagliatelle', 'chicken-cut-into-stripes-only-meat', 'cookies', 'sun-dried-tomatoe', 'bread-ticino', 'semi-hard-cheese', 'margarine', 'porridge-prepared-with-partially-skimmed-milk', 'soya-drink-soy-milk', 'juice-multifruit', 'popcorn-salted', 'chocolate-filled', 'milk-chocolate', 'bread-fruit', 'mix-of-dried-fruits-and-nuts', 'corn', 'tete-de-moine', 'dates', 'pistachio', 'celery', 'white-radish', 'oat-milk', 'cream-cheese', 'bread-rye', 'witloof-chicory', 'apple-crumble', 'goat-cheese-soft', 'grapefruit-pomelo', 'risotto-with-mushrooms-cooked', 'blue-mould-cheese', 'biscuit-with-butter', 'guacamole', 'pecan-nut', 'tofu', 'cordon-bleu-from-pork-schnitzel-fried', 'paprika-chips', 'quinoa', 'kefir-drink', 'm-m-s', 'salad-rocket', 'bread-spelt', 'pizza-with-ham-with-mushrooms-baked', 'fruit-coulis', 'plums', 'beef-minced-only-meat', 'pizza-with-ham-baked', 'pineapple', 'soup-tomato', 'cheddar', 'tea-fruit', 'rice-jasmin', 'seeds', 'focaccia', 'milk', 'coleslaw-chopped-without-sauce', 'pastry-flaky', 'curd', 'savoury-puff-pastry-stick', 'sweet-potato', 'chicken-leg', 'croissant', 'sour-cream', 'ham-turkey', 'processed-cheese', 'fruit-compotes', 'cheesecake', 'pasta-tortelloni-stuffing', 'sauce-cocktail', 'croissant-with-chocolate-filling', 'pumpkin-seeds', 'artichoke', 'champagne', 'grissini', 'sweets-candies', 'brie', 'wienerli-swiss-sausage', 'syrup-diluted-ready-to-drink', 'apple-pie', 'white-bread-with-butter-eggs-and-milk', 'savoury-puff-pastry', 'anchovies', 'tuna-in-oil-drained', 'lemon-pie', 'meat-terrine-pate', 'coriander', 'falafel-balls', 'berries', 'latte-macchiato-with-caffeine', 'faux-mage-cashew-vegan-chers', 'beans-white', 'sugar-melon', 'mixed-seeds', 'hamburger', 'hamburger-bun', 'oil-vinegar-salad-dressing', 'soya-yaourt-yahourt-yogourt-ou-yoghourt', 'chocolate-milk-chocolate-drink', 'celeriac', 'chocolate-mousse', 'cenovis-yeast-spread', 'thickened-cream-35', 'meringue', 'lamb-chop', 'shrimp-prawn-large', 'beef', 'lemon', 'croque-monsieur', 'chives', 'chocolate-cookies', 'birchermuesli-prepared-no-sugar-added', 'fish-crunchies-battered', 'muffin', 'savoy-cabbage-steamed-without-addition-of-salt', 'pine-nuts', 'chorizo', 'chia-grains', 'frying-sausage', 'french-pizza-from-alsace-baked', 'chocolate', 'cooked-sausage', 'grits-polenta-maize-flour', 'gummi-bears-fruit-jellies-jelly-babies-with-fruit-essence', 'wine-rose', 'coca-cola', 'raspberries', 'roll-with-pieces-of-chocolate', 'goat-average-raw', 'lemon-cake', 'coconut-milk', 'rice-wild', 'gluten-free-bread', 'pearl-onions', 'buckwheat-pancake', 'bread-5-grain', 'light-beer', 'sugar-glazing', 'tzatziki', 'butter-herb', 'ham-croissant', 'corn-crisps', 'lentils-green-du-puy-du-berry', 'cocktail', 'rice-whole-grain', 'veal-sausage', 'cervelat', 'sorbet', 'aperitif-with-alcohol-aperol-spritz', 'dips', 'corn-flakes', 'peas', 'tiramisu', 'apricots', 'cake-marble', 'lamb', 'lasagne-meat-prepared', 'coca-cola-zero', 'cake-salted', 'dough-puff-pastry-shortcrust-bread-pizza-dough', 'rice-waffels', 'sekt', 'brioche', 'vegetable-au-gratin-baked', 'mango-dried', 'processed-meat-charcuterie', 'mousse', 'sauce-sweet-sour', 'basil', 'butter-spread-puree-almond', 'pie-apricot-baked-with-cake-dough', 'rusk-wholemeal', 'beef-roast', 'vanille-cream-cooked-custard-creme-dessert', 'pasta-in-conch-form', 'nuts', 'sauce-carbonara', 'fig-dried', 'pasta-in-butterfly-form-farfalle', 'minced-meat', 'carrot-steamed-without-addition-of-salt', 'ebly', 'damson-plum', 'shoots', 'bouquet-garni', 'coconut', 'banana-cake', 'waffle', 'apricot-dried', 'sauce-curry', 'watermelon-fresh', 'sauce-sweet-salted-asian', 'pork-roast', 'blackberry', 'smoked-cooked-sausage-of-pork-and-beef-meat-sausag', 'bean-seeds', 'italian-salad-dressing', 'white-asparagus', 'pie-rhubarb-baked-with-cake-dough', 'tomato-stewed-without-addition-of-fat-without-addition-of-salt', 'cherries', 'nectarine')

In [10]:
args = {}
args['num_classes'] = len(CLASSES)
args['input_size'] = 224
args['num_workers'] = 0
args['num_queries'] = 100
args['null_class_coef'] = 0.5
args['bs'] = 8
args['lr'] = 1e-3
args['epochs'] = 10

In [11]:
def get_transforms(is_train=True):
    if is_train:
        return A.Compose([
                        A.OneOf([
                            A.HueSaturationValue(
                                hue_shift_limit=0.2, 
                                sat_shift_limit= 0.2, 
                                val_shift_limit=0.2, 
                                p=0.9),
                            A.RandomBrightnessContrast(
                                brightness_limit=0.2, 
                                contrast_limit=0.2, 
                                p=0.9)]
                            ,p=0.9),
            
                        A.ToGray(p=0.01),
                    
                        A.HorizontalFlip(p=0.2),
                      
                        A.VerticalFlip(p=0.2),

                        A.Resize(height=args['input_size'], width=args['input_size'], p=1),

                        A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
                      
                        ToTensorV2(p=1.0)],
                      
                      p=1.0,
                     
                      bbox_params=A.BboxParams(format='coco',
                                               min_area=0, 
                                               min_visibility=0,
                                               label_fields=['labels'])
                      )
    else:
        return A.Compose([A.Resize(height=args['input_size'], 
                                   width=args['input_size'], 
                                   p=1.0),
                      ToTensorV2(p=1.0)], 
                      p=1.0, 
                      bbox_params=A.BboxParams(format='coco',
                                               min_area=0, 
                                               min_visibility=0,
                                               label_fields=['labels'])
                      )    

In [12]:
# Reading the annotation files
with open(TRAIN_ANNOTATIONS_PATH) as f:
    train_annotations_data = json.load(f)
    
with open(VAL_ANNOTATIONS_PATH) as f:
    val_annotations_data = json.load(f)

In [13]:
def get_label_id(cats):
    return [CLASSES.index(item) for item in cats]

In [14]:
def ifnone(a, b): return b if a is None else a
def get_annotations(fname, prefix=None):
    "Open a COCO style json in `fname` and returns the lists of filenames (with maybe `prefix`) and labelled bboxes."
    annot_dict = json.load(open(fname))
    id2images, id2bboxes, id2cats = {}, collections.defaultdict(list), collections.defaultdict(list)
    classes = {o['id']:o['name'] for o in annot_dict['categories']}
    for o in annot_dict['annotations']:
        bb = o['bbox']
        id2bboxes[o['image_id']].append([bb[0],bb[1], bb[2], bb[3]])
        id2cats[o['image_id']].append(classes[o['category_id']])
    id2images = {o['id']:ifnone(prefix, '') + o['file_name'] for o in annot_dict['images'] if o['id'] in id2bboxes}
    ids = list(id2images.keys())
    return [id2images[k] for k in ids], [id2bboxes[k] for k in ids], [get_label_id(id2cats[k]) for k in ids]

In [15]:
imgs_train, bboxes_train, cat_train = get_annotations(TRAIN_ANNOTATIONS_PATH) 
df_train = pd.DataFrame([])
df_train['fnames'] = imgs_train
df_train['bboxes'] = bboxes_train
df_train['labels'] = cat_train

imgs_valid, bboxes_valid, cat_valid = get_annotations(VAL_ANNOTATIONS_PATH) 
df_valid = pd.DataFrame([])
df_valid['fnames'] = imgs_valid
df_valid['bboxes'] = bboxes_valid
df_valid['labels'] = cat_valid

In [16]:
df_train.head()

,fnames,bboxes,labels
0,131094.jpg,"[[39.5, 39.5, 167.0, 92.0], [8.2599609375, 8.2...","[5, 6, 7, 0, 8, 9]"
1,131097.jpg,"[[44.47625, 44.47625, 311.3225, 257.0825], [80...","[11, 12, 5]"
2,131098.jpg,"[[58.161249999999995, 58.161249999999995, 256....","[12, 5]"
3,131100.jpg,"[[56.68375000000002, 76.73375, 288.36249999999...","[12, 14, 15, 5, 16]"
4,131101.jpg,"[[122.5, 122.5, 195.0, 128.0]]",[17]


In [17]:
class FoodDataset(Dataset):
    def __init__(self, path, dataframe, transforms=None):
        self.path = path
        self.dataframe = dataframe
        self.transforms = transforms
        
        
    def __len__(self) -> int:
        return len(self.dataframe)
    
    def __getitem__(self,index):
        image_id = self.dataframe['fnames'].iloc[index]
        image_path = os.path.join(self.path, image_id)
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = np.array(image).astype(np.float32)
        image /= 255.0
        # DETR takes in data in coco format 
        boxes = self.dataframe['bboxes'].iloc[index]
        
        
        boxes = np.array(boxes)
        #Area of bb
        area = boxes[:,2] * boxes[:,3]
#         print(area)
        area = torch.as_tensor(area, dtype=torch.float32)
        # AS pointed out by PRVI It works better if the main class is labelled as zero
        labels =  self.dataframe['labels'].iloc[index]
        try:
            if self.transforms:
                sample = {
                    'image': image,
                    'bboxes': boxes,
                    'labels': labels
                }
                sample = self.transforms(**sample)
                image = sample['image']
                boxes = sample['bboxes']
                labels = sample['labels']


            #Normalizing BBOXES

            _,h,w = image.shape
            boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
            target = {}

            target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
            target['labels'] = torch.as_tensor(labels,dtype=torch.long)
            target['image_id'] = torch.tensor([index])
            target['area'] = area

            image_id = image_id.split('.')[0]
            image = image.type(torch.FloatTensor)
            return image, target, image_id
        except Exception as e:
            # return image_id
            # print(str(e))
            print(image_id)
            return image, '_', image_id 

def collate_fn(batch):
    return tuple(zip(*batch))

In [18]:
ds_train = FoodDataset(TRAIN_IMAGE_DIRECTIORY, df_train, transforms = get_transforms(True))
ds_valid = FoodDataset(VAL_IMAGE_DIRECTIORY, df_valid, transforms = get_transforms(False))

../../albumentations\albumentations\augmentations\dropout\cutout.py:52: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  FutureWarning,


In [19]:
dl_train = DataLoader(
    ds_train,
    batch_size = args['bs'],
    shuffle=False,
    num_workers=args['num_workers'],
    collate_fn=collate_fn
    )

dl_valid = DataLoader(
    ds_valid,
    batch_size = args['bs'],
    shuffle=False,
    num_workers=args['num_workers'],
    collate_fn=collate_fn
    )

In [20]:
# list_error = []
for (x,t,i) in tqdm(dl_train):
    # print(i)
    # list_error.append(i)
    continue
#     break

100%|██████████████████████████████████████████████████████████| 4995/4995 [49:16<00:00,  1.69it/s]


In [21]:
break

SyntaxError: 'break' outside loop (668683560.py, line 4)

In [ ]:
for (x,t,i) in tqdm(dl_valid):
    # print(i)
    # list_error.append(i)
    continue

In [ ]:
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

In [ ]:
matcher = HungarianMatcher()

weight_dict = weight_dict = {'loss_ce': 1., 'loss_bbox': 1., 'loss_giou': 1.}

losses = ['labels', 'boxes', 'cardinality']

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_one(data_loader,model,criterion,optimizer, args, device,scheduler,epoch):
    model.train()
    criterion.train()
    
    summary_loss = AverageMeter()
    
    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for step, (images, targets, image_ids) in enumerate(tk0):
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        

        output = model(images)
        
        loss_dict = criterion(output, targets)
        weight_dict = criterion.weight_dict
        
        losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        
        optimizer.zero_grad()

        losses.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        
        summary_loss.update(losses.item(),args['bs'])
        tk0.set_postfix(loss=summary_loss.avg)
        
    return summary_loss

def eval_one(data_loader, model,criterion, args, device):
    model.eval()
    criterion.eval()
    summary_loss = AverageMeter()
    
    with torch.no_grad():
        
        tk0 = tqdm(data_loader, total=len(data_loader))
        for step, (images, targets, image_ids) in enumerate(tk0):
            
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            output = model(images)
        
            loss_dict = criterion(output, targets)
            weight_dict = criterion.weight_dict
        
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
            
            summary_loss.update(losses.item(), args['bs'])
            tk0.set_postfix(loss=summary_loss.avg)
    
    return summary_loss

In [ ]:
device = torch.device('cuda')
model = DETRModel(num_classes=args['num_classes'],num_queries=args['num_queries'])
model = model.to(device)
criterion = SetCriterion(args['num_classes']-1, matcher, weight_dict, eos_coef = args['null_class_coef'], losses=losses)
criterion = criterion.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=args['lr'])

In [ ]:
# model(list([item.to(device) for item in x]))

In [ ]:
for epoch in range(args['epochs']):
    train_loss = train_one(dl_train, model,criterion, optimizer,args, device,scheduler=None,epoch=epoch)
    valid_loss = eval_one(dl_valid, model,criterion, args, device)

    print('|EPOCH {}| TRAIN_LOSS {}| VALID_LOSS {}|'.format(epoch+1,train_loss.avg,valid_loss.avg))